# Tratamento de Dados para RNA

Neste notebook, vamos realizar o tratamento e transformação dos dados para prepará-los para a utilização em uma Rede Neural Artificial (RNA). Atualmente, os dados já passaram por uma primeira etapa de transformação no arquivo TransformTables.ipynb, onde foram preparados para exploração.

## Estrutura dos Dados Atuais

Os dados atuais consistem em 8 conjuntos de dados JSON separados por 4 aparelhos e 2 dias. Cada conjunto de dados contém informações de localização e outros parâmetros obtidos dos dispositivos. Abaixo está um exemplo de um desses conjuntos de dados:

```json
{
    "gnss_data": {
        "latitude": -25.315828333333336,
        "longitude": -54.059338333333336,
        "altitude": 432.5,
        "accuracy": 3.799999952316284,
        "speed": 0.13640138506889343,
        "bearing": 353.6199951171875,
        "provider": "gps",
        "timestamp": 0.33541666666666664,
        "pontoselecionado": "Ponto 1",
        "latitudereal": "-25.315870288",
        "longitudereal": "-54.059354844",
        "gnss_data_json": [
            {
                "svid": 21,
                "constellationType": 1,
                "azimuthDegrees": 205,
                ...
            },
            ...
        ],
        "latitudeDiferencaMetros": 1.7653363172430545,
        "longitudeDiferencaMetros": 4.613154257647693
    }
}
```
Neste exemplo, temos informações de localização (latitude, longitude, altitude), velocidade, precisão, entre outros. Além disso, há uma lista de dados GNSS JSON que contém informações sobre satélites, como ID, tipo de constelação, azimute, e elevação.

Nos próximos passos, vamos realizar as transformações necessárias para ajustar esses dados para a utilização em uma RNA.


______________



__________


# ***APLICAÇÃO***

____________________
________________________



## Importância das Bibliotecas

### Pandas (`pd`)
O Pandas é uma biblioteca Python amplamente utilizada para manipulação e análise de dados. Ele fornece estruturas de dados flexíveis e eficientes, como DataFrame, que facilitam o trabalho com conjuntos de dados tabulares, como os dados que estamos lidando. Com o Pandas, podemos ler, manipular e transformar facilmente os dados em formatos como CSV, JSON e Excel.

### NumPy (`np`)
NumPy é uma biblioteca fundamental para computação científica em Python. Ela oferece suporte para arrays multidimensionais e funções matemáticas que são essenciais para muitas operações de processamento de dados. Em nosso caso, podemos usar o NumPy para realizar operações numéricas eficientes em nossos dados, especialmente quando lidamos com arrays de números, como as coordenadas de latitude e longitude.

### JSON
A biblioteca padrão JSON de Python fornece suporte para trabalhar com dados no formato JSON. Como nossos dados estão em formato JSON, precisamos desta biblioteca para carregar e manipular esses dados em Python. Com a biblioteca JSON, podemos facilmente converter os dados JSON em estruturas de dados Python, como dicionários e listas, para que possamos acessar e manipular suas informações.

### SQLite3
SQLite3 é um módulo Python que permite interagir com bancos de dados SQLite usando SQL. Embora não esteja diretamente relacionado ao tratamento de dados em formatos como JSON, pode ser útil para armazenar e consultar dados em um banco de dados relacional, se necessário.

### OS
O módulo `os` fornece uma maneira de trabalhar com funcionalidades dependentes do sistema operacional. Embora não seja diretamente relacionado ao tratamento de dados, pode ser útil para realizar operações como listar arquivos em um diretório, manipular caminhos de arquivos, etc.

### Datetime
O módulo `datetime` é utilizado para trabalhar com datas e horas em Python. Neste contexto, podemos usá-lo para realizar operações de conversão de timestamps, manipulação de datas e horas em nossos dados, se necessário.

### UTM
O pacote `utm` oferece funções para converter coordenadas de latitude e longitude para o sistema de coordenadas UTM (Universal Transverse Mercator) e vice-versa. Essa funcionalidade é útil para realizar cálculos de distância, área e outras operações baseadas em coordenadas.

Essas bibliotecas são essenciais para lidar eficientemente com os dados que temos e realizar as transformações necessárias para prepará-los para a utilização em nossa Rede Neural Artificial.

In [1]:
# Importação das bibliotecas necessárias
import pandas as pd
import numpy as np
import json
import sqlite3
import os
import datetime
import utm

## Processo de Combinação dos Bancos de Dados SQLite

### Motivação
O objetivo deste processo é combinar vários bancos de dados SQLite em um único banco de dados, simplificando assim o processo de manipulação e processamento dos dados para posterior análise ou treinamento de uma Rede Neural Artificial (RNA). Isso elimina a necessidade de lidar com múltiplos bancos de dados separadamente, tornando o processo mais eficiente e organizado.

### Processo de Combinação
1. **Listagem dos Bancos de Dados:** Primeiro, listamos os nomes dos bancos de dados SQLite que queremos combinar, armazenando-os em uma lista.

2. **Leitura e Combinação dos Dados:** Em seguida, iteramos sobre cada banco de dados separadamente. Para cada banco de dados, extraímos o esquema das tabelas e copiamos os dados para o novo banco de dados. 

3. **Salvando os Dados Combinados:** Por fim, salvamos as alterações e fechamos a conexão com o novo banco de dados, resultando em um único banco de dados contendo todos os dados combinados.

### Vantagens
- Simplifica o processo de manipulação de dados, eliminando a necessidade de lidar com múltiplos bancos de dados.
- Facilita a análise e o processamento dos dados, tornando-os acessíveis em um único banco de dados.
- Organiza os dados de forma mais eficiente, preparando-os para análises posteriores ou treinamento de modelos de Machine Learning.

Este processo é útil em cenários onde múltiplos conjuntos de dados precisam ser combinados para formar um único conjunto de dados coeso e pronto para análise ou modelagem.


In [3]:
# Crie um novo banco de dados
novo_banco_dados = "AllDataBase.db"
conexao = sqlite3.connect(novo_banco_dados)
cursor = conexao.cursor()

# Itere sobre os 8 bancos de dados separados
bancos_de_dados_separados = ["1.db", "2.db", "3.db", "4.db", "5.db", "6.db", "7.db", "8.db"]
for banco in bancos_de_dados_separados:
    # Conecte-se ao banco de dados separado
    conexao_separada = sqlite3.connect(banco)
    cursor_separado = conexao_separada.cursor()

    # Extraia o esquema da tabela
    cursor_separado.execute("SELECT name, sql FROM sqlite_master WHERE type='table'")
    tabelas = cursor_separado.fetchall()

    # Copie os dados da tabela para o novo banco de dados
    for tabela in tabelas:
        tabela_nome, tabela_sql = tabela
        # Verifique se o nome da tabela não é "sqlite_sequence"
        if tabela_nome != "sqlite_sequence":
            dados = cursor_separado.execute("SELECT * FROM {}".format(tabela_nome)).fetchall()
            cursor.execute("CREATE TABLE IF NOT EXISTS {} ({})".format(tabela_nome, tabela_sql.split("(")[1].replace(")", "")))
            cursor.executemany("INSERT INTO {} VALUES ({})".format(tabela_nome, ','.join(['?']*len(dados[0]))), dados)

    # Feche a conexão com o banco de dados separado
    conexao_separada.close()

# Salve as alterações e feche a conexão com o novo banco de dados
conexao.commit()
conexao.close()


## Processo de Extração e Salvamento dos Dados do Banco de Dados SQLite

### Motivação
O objetivo deste processo é extrair os dados de um banco de dados SQLite e salvá-los em um arquivo JSON. Isso facilita a manipulação e o compartilhamento dos dados, permitindo que sejam utilizados em diferentes contextos, como análise de dados, treinamento de modelos, entre outros.

### Processo de Extração e Salvamento
1. **Conexão ao Banco de Dados SQLite:** Primeiro, estabelecemos uma conexão com o banco de dados SQLite utilizando a biblioteca `sqlite3`. Isso nos permite acessar e manipular os dados contidos no banco de dados.

2. **Extração dos Dados das Tabelas:** Em seguida, iteramos sobre as tabelas do banco de dados utilizando uma consulta SQL para obter os nomes das tabelas. Para cada tabela, executamos outra consulta para extrair todos os dados contidos nela.

3. **Conversão dos Dados em Formato JSON:** Para cada linha de dados obtida, criamos um dicionário Python contendo os dados da linha, utilizando as descrições das colunas fornecidas pelo cursor. Em seguida, adicionamos este dicionário à lista `dados_json`.

4. **Salvamento dos Dados em um Arquivo JSON:** Por fim, salvamos a lista de dicionários `dados_json` em um arquivo JSON utilizando a função `json.dump()`. Isso cria um arquivo JSON contendo todos os dados extraídos do banco de dados, facilitando seu uso posterior.

### Vantagens
- Facilita a manipulação e o compartilhamento dos dados, pois eles são armazenados em um formato amplamente utilizado e interoperável.
- Permite o uso dos dados em diferentes contextos e ferramentas que suportam o formato JSON.
- Automatiza o processo de extração e salvamento de dados, tornando-o mais eficiente e menos propenso a erros.

Este processo é útil em cenários onde é necessário extrair dados de um banco de dados SQLite e utilizá-los em diferentes contextos de análise ou modelagem.


In [6]:
# Conecte-se ao banco de dados SQLite
conexao = sqlite3.connect("AllDataBase.db")
cursor = conexao.cursor()

# Lista para armazenar os dados convertidos
dados_json = []

# Itere sobre as tabelas e extraia os dados
cursor.execute("SELECT name FROM sqlite_master WHERE type='table'")
tabelas = cursor.fetchall()
for tabela in tabelas:
    nome_tabela = tabela[0]
    cursor.execute("SELECT * FROM {}".format(nome_tabela))
    dados_tabela = cursor.fetchall()

    # Crie um dicionário com os dados da tabela e adicione à lista
    for linha in dados_tabela:
        dicionario_linha = dict(zip([coluna[0] for coluna in cursor.description], linha))
        dados_json.append({nome_tabela: dicionario_linha})

# Salve os dados em um arquivo JSON
with open("dados.json", "w") as arquivo_json:
    json.dump(dados_json, arquivo_json, indent=4)

# Feche a conexão com o banco de dados SQLite
conexao.close()


## Processo de Limpeza do Campo 'gnss_data_json' no Arquivo JSON

### Motivação
O campo 'gnss_data_json' nos dados JSON contém informações sobre satélites em formato de string JSON. No entanto, para facilitar a manipulação e análise dos dados, é desejável converter essas strings JSON em objetos Python e reorganizar os dados conforme necessário.

### Processo de Limpeza
1. **Carregamento dos Dados JSON:** Primeiramente, carregamos os dados do arquivo JSON especificado.

2. **Limpeza do Campo 'gnss_data_json':** Iteramos sobre os dados e verificamos se o campo 'gnss_data_json' está presente e é uma string. Se sim, convertemos essa string em um objeto JSON, extraímos a lista de satélites e atualizamos os dados, removendo o campo 'gnss_data_json'.

3. **Salvamento dos Dados Limpos:** Por fim, salvamos os dados limpos de volta no arquivo JSON, sobrescrevendo o arquivo original.

### Vantagens
- Facilita a manipulação e análise dos dados, convertendo as strings JSON em objetos Python.
- Organiza os dados de uma forma mais acessível e estruturada, facilitando seu uso em análises posteriores ou treinamento de modelos.

Este processo é útil para preparar os dados para análises subsequentes ou uso em modelos de Machine Learning, tornando-os mais adequados para manipulação e processamento.


In [2]:
import json

# Função para limpar o campo 'gnss_data_json'
def limpar_gnss_data_json(dados):
    for dado in dados:
        if 'gnss_data' in dado:
            gnss_data = dado['gnss_data']
            # Verificar se 'gnss_data_json' está presente e é uma string
            if 'gnss_data_json' in gnss_data and isinstance(gnss_data['gnss_data_json'], str):
                # Converter a string em um objeto JSON
                gnss_data_json_objeto = json.loads(gnss_data['gnss_data_json'])
                # Atualizar o campo 'gnss_data_json' com a lista de satélites
                gnss_data['satellites'] = gnss_data_json_objeto.get('satellites', [])
                # Remover o campo 'gnss_data_json'
                del gnss_data['gnss_data_json']

# Nome do arquivo JSON
arquivo_json = "dados.json"

# Abrir o arquivo JSON e carregar os dados
with open(arquivo_json, "r") as arquivo:
    dados = json.load(arquivo)

# Limpar o campo 'gnss_data_json'
limpar_gnss_data_json(dados)

# Sobrescrever o arquivo JSON com os dados limpos
with open(arquivo_json, "w") as arquivo_json_limpo:
    json.dump(dados, arquivo_json_limpo, indent=4)


## Processo de Conversão de Timestamps no Arquivo JSON

### Motivação
Os timestamps nos dados JSON estão em milissegundos desde a Epoch (1 de janeiro de 1970). Para facilitar a análise temporal dos dados, é útil converter esses timestamps para uma representação mais compreensível, como minutos normalizados desde o início do dia.

### Processo de Conversão
1. **Carregamento dos Dados JSON:** Os dados são carregados do arquivo JSON especificado.

2. **Conversão dos Timestamps:** Iteramos sobre cada entrada dos dados e verificamos se há um campo 'timestamp' presente. Se houver, usamos a função `converter_timestamp` para converter o timestamp para minutos normalizados desde o início do dia.

3. **Atualização e Salvamento dos Dados:** Os timestamps convertidos são atualizados nos dados e os dados modificados são salvos de volta no mesmo arquivo JSON.

### Vantagens
- Facilita a análise temporal dos dados, convertendo os timestamps em uma representação mais compreensível.
- Normaliza os timestamps para um intervalo de 0 a 1, o que pode ser útil em algumas análises ou modelos de Machine Learning.

Este processo é útil para preparar os dados temporais para análises subsequentes ou uso em modelos que requerem uma representação temporal clara dos dados.


In [4]:

def converter_timestamp(timestamp):
    # Converter o timestamp para um objeto datetime
    data_hora = datetime.datetime.fromtimestamp(timestamp / 1000)  # Dividido por 1000 para converter de milissegundos para segundos
    # Calcular o número de minutos desde o início do dia
    minutos_do_dia = data_hora.hour * 60 + data_hora.minute
    # Normalizar os minutos do dia para o intervalo de 0 a 1
    minutos_normalizados = minutos_do_dia / (24 * 60)  # 24 horas * 60 minutos
    return minutos_normalizados

# Nome do arquivo JSON
arquivo_json = "dados.json"

# Carregar os dados do arquivo JSON
with open(arquivo_json, "r") as arquivo:
    dados_json = json.load(arquivo)

# Converter os timestamps em cada entrada do arquivo JSON
for dado in dados_json:
    if 'gnss_data' in dado:
        gnss_data = dado['gnss_data']
        if 'timestamp' in gnss_data:
            gnss_data['timestamp'] = converter_timestamp(gnss_data['timestamp'])

# Salvar os dados modificados no mesmo arquivo JSON
with open(arquivo_json, "w") as arquivo:
    json.dump(dados_json, arquivo, indent=4)


## Processo de Conversão de Coordenadas para UTM e Cálculo de Diferença

### Motivação
O objetivo deste processo é converter as coordenadas geográficas (latitude e longitude) para o sistema de coordenadas UTM (Universal Transverse Mercator) e calcular a diferença em metros entre as coordenadas coletadas e as coordenadas reais. Isso pode ser útil para análises de precisão de GPS e para entender a exatidão das medições.

### Processo de Conversão e Cálculo
1. **Carregamento dos Dados JSON:** Os dados são carregados do arquivo JSON especificado.

2. **Iteração sobre os Dados:** Iteramos sobre cada entrada dos dados e verificamos se há um campo 'gnss_data' presente.

3. **Conversão para UTM:** As coordenadas coletadas e as coordenadas reais são convertidas para o sistema de coordenadas UTM usando a função `utm.from_latlon()`.

4. **Cálculo da Diferença:** Calculamos a diferença em metros entre as coordenadas coletadas e as coordenadas reais.

5. **Atualização dos Valores:** Os valores calculados de diferença são adicionados ao dicionário de dados.

6. **Salvamento dos Dados Modificados:** Os dados ajustados são salvos em um novo arquivo JSON.

### Vantagens
- Permite uma análise mais precisa da exatidão das medições de GPS.
- Facilita a compreensão das diferenças entre as coordenadas coletadas e as coordenadas reais.

Este processo é útil para entender a precisão dos dados de localização e para identificar possíveis desvios nas medições GPS.


In [8]:
# Função para converter strings para float
def converter_para_float(valor):
    try:
        return float(valor)
    except ValueError:
        return None

# Nome do arquivo JSON
arquivo_json = "dados.json"

# Abrir o arquivo JSON original
with open(arquivo_json, "r") as arquivo:
    dados = json.load(arquivo)

# Iterar sobre os dados
for dado in dados:
    if 'gnss_data' in dado:
        gnss_data = dado['gnss_data']
        # Converter coordenadas coletadas para UTM
        latitude_coletada = converter_para_float(gnss_data['latitude'])
        longitude_coletada = converter_para_float(gnss_data['longitude'])
        if latitude_coletada is not None and longitude_coletada is not None:
            latitude_utm_coletada, longitude_utm_coletada, _, _ = utm.from_latlon(latitude_coletada, longitude_coletada)
        
            # Converter coordenadas reais para UTM
            latitudereal = converter_para_float(gnss_data['latitudereal'])
            longitudereal = converter_para_float(gnss_data['longitudereal'])
            if latitudereal is not None and longitudereal is not None:
                latitude_utm_real, longitude_utm_real, _, _ = utm.from_latlon(latitudereal, longitudereal)
        
                # Calcular a diferença em metros
                latitude_diferenca_m = latitude_utm_coletada - latitude_utm_real
                longitude_diferenca_m = longitude_utm_coletada - longitude_utm_real
        
                # Atualizar os valores no dicionário
                gnss_data['latitudeDiferencaMetros'] = latitude_diferenca_m
                gnss_data['longitudeDiferencaMetros'] = longitude_diferenca_m

# Salvar os dados ajustados em um novo arquivo JSON
with open(arquivo_json, "w") as arquivo_json_diferenca:
    json.dump(dados, arquivo_json_diferenca, indent=4)


## Leitura de Dados e Extração de "svid" de Arquivos JSON

### Motivação
O objetivo deste processo é extrair os valores de "svid" dos arquivos JSON para identificar todos os identificadores únicos de satélites presentes nos dados.

### Passos do Processo
1. **Abertura dos Arquivos JSON:** Primeiramente, abrimos os arquivos `dados.json` para acessar seus conteúdos e realizar a extração dos dados.

2. **Extração de "svid":** Iteramos sobre os dados de cada arquivo JSON e buscamos o campo "satellites". Dentro desse campo, encontramos o valor de "svid" para cada satélite listado.

3. **Construção da Lista de "svid":** Armazenamos todos os valores de "svid" encontrados em uma lista, garantindo que não haja duplicatas.

4. **Ordenação da Lista:** Ordenamos a lista de valores únicos de "svid" para facilitar a visualização e identificação dos IDs de satélites presentes nos dados.

5. **Exibição da Lista:** Exibimos a lista de valores únicos de "svid", destacando todos os IDs de satélites presentes nos dados.

### Vantagens
- Identificação rápida e eficiente de todos os identificadores únicos de satélites presentes nos dados.
- Facilita a análise e a interpretação dos dados relacionados aos satélites utilizados em cada ponto de medição.
- Permite uma melhor compreensão da cobertura e distribuição dos satélites ao longo do tempo e espaço.

Este processo é útil para obter insights sobre os satélites envolvidos na coleta dos dados e pode auxiliar na análise e no planejamento de sistemas de posicionamento global.


In [5]:
# Lista para armazenar os valores únicos de 'svid'
valores_svid = set()  # Usaremos um conjunto para garantir valores únicos

# Abrir o arquivo JSON e carregar os dados
with open("dados.json", "r") as arquivo_json:
    dados = json.load(arquivo_json)

    # Iterar sobre os dados e extrair os valores únicos de 'svid'
    for dado in dados:
        if 'gnss_data' in dado and 'satellites' in dado['gnss_data']:
            for satelite in dado['gnss_data']['satellites']:
                svid = satelite.get('svid')
                if svid is not None:
                    valores_svid.add(svid)

# Converter o conjunto para uma lista para manter a ordem
valores_svid = list(valores_svid)

# Imprimir os valores únicos de 'svid'
print("Valores únicos de 'svid':", valores_svid)


Valores únicos de 'svid': [128, 129, 2, 3, 4, 5, 131, 6, 8, 135, 10, 11, 12, 138, 14, 130, 9, 17, 18, 19, 20, 21, 15, 16, 24, 25, 26, 27, 28, 23, 133, 31, 32, 33, 34, 134, 1, 36, 7, 30, 37, 41, 42, 43, 44, 136, 46, 45, 35, 137, 57, 13, 29, 22, 132, 120, 121, 122, 123, 124, 125, 126, 127]


## Valores Únicos de 'svid'

Os valores listados acima representam os IDs únicos de satélites ('svid') encontrados nos dados. Cada ID está associado a um satélite específico no sistema de navegação por satélite (GNSS).

Cada satélite é identificado por um ID único que é atribuído a ele. Por exemplo, o ID 21 pode corresponder a um satélite GPS específico.

## Tratamento dos IDs 'svid'

Para tratar esses IDs, vamos realizar as seguintes etapas:

1. **Criação de Campos Específicos:** Criaremos campos separados para cada ID de satélite ('svid') encontrado nos dados. Esses campos serão utilizados para indicar a presença ou ausência de cada satélite.
   
2. **Atribuição de Valores:** Se um satélite estiver presente em um determinado ponto de dados, o campo correspondente ao seu ID ('svid') será preenchido com o valor 1; caso contrário, será preenchido com o valor 0.

3. **Extração de Informações:** Além disso, vamos extrair informações relevantes de cada satélite, como 'azimuthDegrees', 'elevationDegrees', 'cn0DbHz', etc., e utilizá-las para preencher esses campos.

Agora, vamos proceder com a transformação dos dados.


In [5]:
# Carregar os dados do arquivo JSON original
with open("dados.json", "r") as arquivo_json:
    dados = json.load(arquivo_json)

# Conjunto para armazenar os tipos únicos de constelação
constellation_types = set()

# Iterar sobre cada item no JSON original
for item in dados:
    gnss_data = item.get('gnss_data', {})
    satellites = gnss_data.get('satellites', [])

    # Iterar sobre cada satélite
    for satellite in satellites:
        constellation_type = satellite.get('constellationType')

        # Verificar se o tipo de constelação está presente nos dados do satélite
        if constellation_type is not None:
            # Adicionar o tipo de constelação ao conjunto
            constellation_types.add(constellation_type)

# Armazenar os tipos únicos de constelação em uma lista
unique_constellation_types = list(constellation_types)

# Imprimir os tipos únicos de constelação encontrados
print("Tipos únicos de constelação:")
for constellation_type in unique_constellation_types:
    print(constellation_type)


Tipos únicos de constelação:
1
2
3
5
6


In [7]:
import json

# Carregar os dados do arquivo JSON original
with open("dados.json", "r") as arquivo_json:
    dados = json.load(arquivo_json)

# Tipos únicos de constelação
unique_constellation_types = [1, 2, 3, 5, 6]

# Dicionário para armazenar os satélites agrupados por tipo de constelação
constellation_satellites = {constellation_type: [] for constellation_type in unique_constellation_types}

# Iterar sobre cada item no JSON original
for item in dados:
    gnss_data = item.get('gnss_data', {})
    satellites = gnss_data.get('satellites', [])

    # Iterar sobre cada satélite
    for satellite in satellites:
        svid = satellite.get('svid')
        constellation_type = satellite.get('constellationType')

        # Verificar se o tipo de constelação está entre os tipos únicos
        if constellation_type in unique_constellation_types:
            # Verificar se o svid já está na lista de satélites correspondente
            if svid not in constellation_satellites[constellation_type]:
                # Adicionar o svid à lista correspondente ao tipo de constelação
                constellation_satellites[constellation_type].append(svid)

# Imprimir os satélites agrupados por tipo de constelação
for constellation_type, satellites in constellation_satellites.items():
    print("Satélites para a constelação", constellation_type, ":", satellites)


Satélites para a constelação 1 : [21, 8, 2, 32, 28, 27, 31, 3, 4, 10, 17, 9, 26, 19, 1, 16, 7, 6, 30, 11, 14, 20, 22, 5, 13, 15]
Satélites para a constelação 2 : [125, 138, 131, 135, 133, 120, 121, 124, 129, 130, 134, 123, 136, 127, 126, 128, 132, 137, 122]
Satélites para a constelação 3 : [19, 5, 4, 18, 20, 3, 6, 14, 15, 21, 16, 9, 22, 10, 23, 8, 11, 1, 12, 2, 24, 13]
Satélites para a constelação 5 : [43, 11, 14, 42, 33, 34, 28, 12, 21, 26, 24, 41, 25, 27, 8, 23, 13, 32, 30, 37, 20, 46, 19, 18, 15, 17, 31, 44, 57, 22, 45, 35]
Satélites para a constelação 6 : [19, 33, 12, 4, 10, 26, 11, 31, 9, 18, 21, 36, 24, 5, 34, 25, 3, 2, 1, 7, 13, 8, 15, 14, 27, 30, 23, 29]


In [9]:
import json

# Carregar os dados do arquivo JSON original
with open("dados.json", "r") as arquivo_json:
    dados = json.load(arquivo_json)

# Tipos únicos de constelação
unique_constellation_types = [1, 2, 3, 5, 6]

# Dicionário para armazenar os satélites agrupados por tipo de constelação
constellation_satellites = {constellation_type: [] for constellation_type in unique_constellation_types}

# Iterar sobre cada item no JSON original
for item in dados:
    gnss_data = item.get('gnss_data', {})
    satellites = gnss_data.get('satellites', [])

    # Iterar sobre cada satélite
    for satellite in satellites:
        svid = satellite.get('svid')
        constellation_type = satellite.get('constellationType')

        # Verificar se o tipo de constelação está entre os tipos únicos
        if constellation_type in unique_constellation_types:
            # Verificar se o svid já está na lista de satélites correspondente
            if svid not in constellation_satellites[constellation_type]:
                # Adicionar o svid à lista correspondente ao tipo de constelação
                constellation_satellites[constellation_type].append(svid)

# Criar um esqueleto inicial para o novo JSON com base nas listas de satélites
novo_json = []

# Iterar sobre cada lista de satélites
for constellation_type, satellites in constellation_satellites.items():
    # Criar um dicionário para representar a constelação no novo JSON
    constellation_data = {
        "gnss_data": {
            "latitude": 0,
            "longitude": 0,
            "altitude": 0,
            "accuracy": 0,
            "speed": 0,
            "bearing": 0,
            "provider": 0,
            "timestamp": 0,
            "pontoselecionado": 0,
            "latitudereal": 0,
            "longitudereal": 0,
            "latitudeDiferencaMetros": 0,
            "longitudeDiferencaMetros": 0,
            f"constellation_{constellation_type}": []
        }
    }

    # Adicionar cada satélite à lista correspondente na constelação
    for svid in satellites:
        satellite_data = {
            "svid": svid,
            "azimuthDegrees": 0,
            "elevationDegrees": 0,
            "cn0DbHz": 0,
            "basebandCn0DbHz": 0,
            "carrierFrequencyHz": 0,
            "exists": 0
        }
        constellation_data["gnss_data"][f"constellation_{constellation_type}"].append(satellite_data)

    # Adicionar os dados da constelação ao novo JSON
    novo_json.append(constellation_data)

# Imprimir o novo JSON populado com 0 quando não houver correspondência
print(json.dumps(novo_json, indent=4))


[
    {
        "gnss_data": {
            "latitude": 0,
            "longitude": 0,
            "altitude": 0,
            "accuracy": 0,
            "speed": 0,
            "bearing": 0,
            "provider": 0,
            "timestamp": 0,
            "pontoselecionado": 0,
            "latitudereal": 0,
            "longitudereal": 0,
            "latitudeDiferencaMetros": 0,
            "longitudeDiferencaMetros": 0,
            "constellation_1": [
                {
                    "svid": 21,
                    "azimuthDegrees": 0,
                    "elevationDegrees": 0,
                    "cn0DbHz": 0,
                    "basebandCn0DbHz": 0,
                    "carrierFrequencyHz": 0,
                    "exists": 0
                },
                {
                    "svid": 8,
                    "azimuthDegrees": 0,
                    "elevationDegrees": 0,
                    "cn0DbHz": 0,
                    "basebandCn0DbHz": 0,
                    "car

In [1]:
import json

# Dados das constelações mapeadas
constelacoes = {
    1: [21, 8, 2, 32, 28, 27, 31, 3, 4, 10, 17, 9, 26, 19, 1, 16, 7, 6, 30, 11, 14, 20, 22, 5, 13, 15],
    2: [125, 138, 131, 135, 133, 120, 121, 124, 129, 130, 134, 123, 136, 127, 126, 128, 132, 137, 122],
    3: [19, 5, 4, 18, 20, 3, 6, 14, 15, 21, 16, 9, 22, 10, 23, 8, 11, 1, 12, 2, 24, 13],
    5: [43, 11, 14, 42, 33, 34, 28, 12, 21, 26, 24, 41, 25, 27, 8, 23, 13, 32, 30, 37, 20, 46, 19, 18, 15, 17, 31, 44, 57, 22, 45, 35],
    6: [19, 33, 12, 4, 10, 26, 11, 31, 9, 18, 21, 36, 24, 5, 34, 25, 3, 2, 1, 7, 13, 8, 15, 14, 27, 30, 23, 29]
}

# Abrir e ler o arquivo dados.json
with open("dados.json", "r") as arquivo:
    dados_fornecidos = json.load(arquivo)

# Novo JSON com a estrutura definida anteriormente
novo_json = []

# Iterar sobre cada item no JSON fornecido
for item in dados_fornecidos:
    gnss_data = item["gnss_data"]
    # Criar um dicionário para representar a unidade no novo JSON
    unidade = {
        "gnss_data": {
            "latitude": gnss_data["latitude"],
            "longitude": gnss_data["longitude"],
            "altitude": gnss_data["altitude"],
            "accuracy": gnss_data["accuracy"],
            "speed": gnss_data["speed"],
            "bearing": gnss_data["bearing"],
            "provider": gnss_data["provider"],
            "timestamp": gnss_data["timestamp"],
            "pontoselecionado": gnss_data["pontoselecionado"],
            "latitudereal": gnss_data["latitudereal"],
            "longitudereal": gnss_data["longitudereal"],
            "latitudeDiferencaMetros": gnss_data["latitudeDiferencaMetros"],
            "longitudeDiferencaMetros": gnss_data["longitudeDiferencaMetros"]
        }
    }

    # Adicionar cada satélite à lista correspondente na constelação
    for constellation_type, satellites in constelacoes.items():
        constellation_key = f"constellation_{constellation_type}"
        if constellation_key not in unidade["gnss_data"]:
            unidade["gnss_data"][constellation_key] = []

        for svid in satellites:
            satellite_found = False
            # Verificar se o satélite está presente nos dados fornecidos
            for satellite in gnss_data["satellites"]:
                if satellite["svid"] == svid:
                    satellite_found = True
                    satellite_data = {
                        "svid": satellite["svid"],
                        "azimuthDegrees": satellite["azimuthDegrees"],
                        "elevationDegrees": satellite["elevationDegrees"],
                        "cn0DbHz": satellite["cn0DbHz"],
                        "basebandCn0DbHz": satellite["basebandCn0DbHz"],
                        "carrierFrequencyHz": satellite["carrierFrequencyHz"],
                        "exists": 1  # Definido como 1, pois o satélite existe neste exemplo
                    }
                    unidade["gnss_data"][constellation_key].append(satellite_data)
                    break

            # Se o satélite não estiver presente, adicionar com os valores padrão
            if not satellite_found:
                satellite_data = {
                    "svid": svid,
                    "azimuthDegrees": 0,
                    "elevationDegrees": 0,
                    "cn0DbHz": 0,
                    "basebandCn0DbHz": 0,
                    "carrierFrequencyHz": 0,
                    "exists": 0
                }
                unidade["gnss_data"][constellation_key].append(satellite_data)

    # Adicionar os dados da unidade ao novo JSON
    novo_json.append(unidade)

# Salvar o novo JSON em um arquivo
with open("novo_dados.json", "w") as arquivo_json:
    json.dump(novo_json, arquivo_json, indent=4)

print("Novo JSON criado e salvo com sucesso!")


Novo JSON criado e salvo com sucesso!


## Processamento de Dados GNSS e Constelações

Este código Python tem como objetivo processar dados GNSS (Sistema Global de Navegação por Satélite) contidos em um arquivo JSON e mapear os satélites presentes em constelações específicas. O resultado é então armazenado em um novo arquivo JSON com a estrutura de dados modificada.

### Funcionamento do Código:

1. **Leitura do arquivo JSON**: O código começa lendo os dados do arquivo "dados.json" que contém informações GNSS.

2. **Mapeamento de Constelações**: As constelações GNSS são mapeadas para identificar os satélites associados a cada uma delas.

3. **Processamento dos Dados GNSS**: Para cada unidade de dados GNSS no arquivo JSON, o código verifica se cada satélite mapeado para uma constelação está presente nos dados fornecidos. Se estiver, os dados do satélite são adicionados ao novo JSON. Caso contrário, são adicionados com valores padrão.

4. **Criação do Novo JSON**: Os dados processados são armazenados em um novo arquivo JSON chamado "novo_dados.json".

### Resultado:

O resultado é um novo arquivo JSON contendo os dados GNSS modificados, onde cada unidade de dados possui informações sobre a localização, precisão e velocidade, além dos satélites presentes em cada constelação mapeada. Satélites ausentes nos dados originais são representados com valores padrão.

O arquivo "novo_dados.json" pode ser usado para análise adicional ou para alimentar sistemas que requerem dados GNSS formatados de uma maneira específica.
